In [ ]:
import tensorflow as tf
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras import layers
from sklearn.metrics import confusion_matrix

In [2]:
SEED        = 123
BATCH_SIZE  = 32
IMG_HEIGHT  = 224
IMG_WIDTH   = 224
IMG_SIZE    = (IMG_HEIGHT, IMG_WIDTH, 3)

In [ ]:
(train_ds, val_ds) = tf.keras.preprocessing.image_dataset_from_directory(
    "datasets/Gastrovision",
    validation_split=0.15,
    subset="both",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

In [4]:
# data augmentation
data_augmentation = Sequential([
    layers.RandomBrightness(factor=0.2, value_range=(0, 255), seed=SEED),
    layers.RandomRotation(factor=0.15),
    layers.RandomFlip(mode="horizontal")
])

In [5]:
# We can use any model that we want
model_base = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

model_base.trainable=True
for layer in model_base.layers[:-4]:
    layer.trainable=False

In [6]:
inputs = keras.Input(shape=IMG_SIZE)
x = data_augmentation(inputs)
x = keras.applications.resnet50.preprocess_input(x)
x = model_base(x)
x = Flatten()(x)
x = Dense(256)(x)
x = Dropout(0.5)(x)
outputs = Dense(units=8, activation='softmax')(x)
model= keras.Model(inputs=inputs, outputs=outputs)

In [7]:
model.compile(
    optimizer="adam",
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

In [8]:
def get_early_stopping():
    return tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

early_stopping = get_early_stopping()

callbacks = [early_stopping]

In [ ]:
history = model.fit(train_ds, validation_data=val_ds, batch_size=BATCH_SIZE, epochs=30, callbacks=callbacks)

In [ ]:
predictions = np.array([])
labels = np.array([])

for x, y in val_ds:
    Y_pred = model.predict(x)
    y_prediction = np.argmax(Y_pred, axis=1)
    predictions = np.concatenate([predictions, y_prediction])
    labels = np.concatenate([labels, y.numpy()])

print(confusion_matrix(y_true=labels, y_pred=predictions))